In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import *

In [ ]:
du=0.00002
dv=0.00001
dx=0.02
dt=1
latticeNumber = 128
time = 2000
one_step_time = 2500

In [ ]:
def GrayScott(f,k,latticeNumber,save_dir):
    stepLoop = int(one_step_time/dt)
    loopNumber = int(time/one_step_time)

    u1 = cp.ones((latticeNumber,latticeNumber))
    v1 = cp.random.rand(latticeNumber,latticeNumber)*0.01

  #initial condition
    center = int(latticeNumber/2)
    for x in range(latticeNumber):
        for y in range(latticeNumber):
            if (x-center)**2+(y-center)**2 < (20+5*randn())**2:
                u1[x,y]=0.5+rand()*0.01-0.005
                v1[x,y]=0.25+rand()*0.01-0.005

    Ku = cp.zeros((latticeNumber,latticeNumber))
    Kv = cp.zeros((latticeNumber,latticeNumber))

    Ku[0,0]=1+4*du/dx/dx*dt
    Ku[1,0]=-du/dx/dx*dt
    Ku[-1,0]=-du/dx/dx*dt
    Ku[0,1]=-du/dx/dx*dt
    Ku[0,-1]=-du/dx/dx*dt

    Kv[0,0]=1+4*dv/dx/dx*dt
    Kv[1,0]=-dv/dx/dx*dt
    Kv[-1,0]=-dv/dx/dx*dt
    Kv[0,1]=-dv/dx/dx*dt
    Kv[0,-1]=-dv/dx/dx*dt

    Kufi=1/cp.fft.fft2(Ku)
    Kvfi=1/cp.fft.fft2(Kv)

    file_name_list = []
    image_list = []
    for t in range(loopNumber):
        for i in range(stepLoop):
            u2 = cp.fft.ifft2(cp.fft.fft2(u1 + dt*(-u1*v1*v1+f*(1-u1)))*Kufi)
            v2 = cp.fft.ifft2(cp.fft.fft2(v1 + dt*(u1*v1*v1-(f+k)*v1))*Kvfi)
            u1 = cp.fft.ifft2(cp.fft.fft2(u2 + dt*(-u2*v2*v2+f*(1-u2)))*Kufi)
            v1 = cp.fft.ifft2(cp.fft.fft2(v2 + dt*(u2*v2*v2-(f+k)*v2))*Kvfi)

        u2 = u1.reshape(latticeNumber*latticeNumber)
        if cp.var(u2)>0.001:
            U = (u1/u1.max())*255
            h = cp.asnumpy(U).astype(np.uint8)

            pil_image = Image.fromarray(h)
            image_list.append(pil_image)
        else:
            return image_list
    return image_list

In [ ]:
image_list = GrayScott(f,k)